**рекуррентная нейронная сеть LSTM**

Евгений Борисов <esborisov@sevsu.ru>

---

In [1]:
import numpy as np
from numpy import random as rng
import matplotlib.pyplot as plt

np.set_printoptions(precision=2) # вывод на печать чисел до 2 знака
# pd.options.display.max_colwidth=200 

----

In [2]:
with open('../data/text/dostoevsky-besy.txt','rt',encoding='utf-8') as f: text = f.read()    
print(len(text))    
text

921


'Праздник состоялся, несмотря ни на какие недоумения прошедшего "Шпигулинского" дня. Я думаю, что если бы даже Лембке умер в ту самую ночь, то праздник все-таки бы состоялся на утро, - до того много соединяла с ним какого-то особенного значения Юлия Михайловна. Увы, она до последней минуты находилась в ослеплении и не понимала настроения общества. Никто под конец не верил, что торжественный день пройдет без какого-нибудь колоссального приключения, без "развязки", как выражались иные, заранее потирая руки. Многие, правда, старались принять самый нахмуренный и политический вид; но вообще говоря, непомерно веселит русского человека всякая общественная скандальная суматоха. Правда, было у нас нечто и весьма посерьезнее одной лишь жажды скандала: было всеобщее раздражение, что-то неутолимо злобное; казалось, всем все надоело ужасно. Воцарился какой-то всеобщий сбивчивый цинизм, цинизм через силу, как бы с натуги.\n'

In [3]:
text = text.lower()[:-1]
abc = { c:i for i,c in enumerate(sorted(set(text))) } 

In [4]:
text

'праздник состоялся, несмотря ни на какие недоумения прошедшего "шпигулинского" дня. я думаю, что если бы даже лембке умер в ту самую ночь, то праздник все-таки бы состоялся на утро, - до того много соединяла с ним какого-то особенного значения юлия михайловна. увы, она до последней минуты находилась в ослеплении и не понимала настроения общества. никто под конец не верил, что торжественный день пройдет без какого-нибудь колоссального приключения, без "развязки", как выражались иные, заранее потирая руки. многие, правда, старались принять самый нахмуренный и политический вид; но вообще говоря, непомерно веселит русского человека всякая общественная скандальная суматоха. правда, было у нас нечто и весьма посерьезнее одной лишь жажды скандала: было всеобщее раздражение, что-то неутолимо злобное; казалось, всем все надоело ужасно. воцарился какой-то всеобщий сбивчивый цинизм, цинизм через силу, как бы с натуги.'

In [5]:
# w=15

# symbol_seq = [ [ text[i:i+w],text[i+w] ] for i in range(len(text)-w) ]
# symbol_seq

In [6]:
# eye = np.eye(max(abc.values())+1)


In [7]:
# import re
# re.split(r'[.,]',text)
# text.split('.')

In [8]:
text_encoded = np.array(list( map( lambda c: abc[c],list(text) ) ))
text_encoded = np.eye(max(abc.values())+1,dtype=np.float32)[text_encoded]

In [9]:
w = 125
text_encoded_seq = [ 
    [ text_encoded[i:i+w],text_encoded[i+w] ] 
    for i in range(len(text_encoded)-w) 
]

In [10]:
X_train = np.stack([ s[0] for s in text_encoded_seq ])
y_train = np.stack([ s[1] for s in text_encoded_seq ])

X_train.shape, y_train.shape

((795, 125, 36), (795, 36))

In [11]:
# X_train

In [12]:
import torch
import torch.nn as nn

In [13]:
class Model(nn.Module):
    
    def __init__(self, input_dim, hid_dim):
        super().__init__()
        #self.rcl0 = nn.GRU(input_dim, hid_dim,)
        self.rcl0 = nn.LSTM(input_dim, hid_dim,)
        self.dense0 = nn.Linear(hid_dim,input_dim) 
        self.softmax0 = nn.Softmax(dim=1)

    def forward(self,x):
        # o, s = self.rcl0(x)
        o, _ = self.rcl0(x)
        o = self.dense0(o[:,-1,:])
        o = self.softmax0(o)
        return o
        #return o[:,-1,:],s[:,-1,:]
        
    def predict(self, x):    
        return self.forward(x)

In [14]:
model = Model(X_train.shape[-1],1024)

In [15]:
# x = torch.Tensor(input_seq[[1],:,:])
# with torch.set_grad_enabled(False): 
#     y = model.forward(x)    
#     #y,s = model.forward(x)
# y.shape

In [16]:
from torch import optim

#criterion = nn.BCELoss() # ф-ция потери
criterion = nn.MSELoss() # ф-ция потери
optimizer = optim.Adam(model.parameters(), lr=1e-3) # метод оптимизации ф-ции потери

In [17]:
# используем GPU если есть
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [18]:
model = model.to(device)

In [19]:
with torch.set_grad_enabled(False):
    loss = criterion( 
            torch.Tensor(y_train).to(device), 
            model.predict( torch.Tensor(X_train).to(device) ) 
        ).cpu().numpy().flatten()[0]
    
loss_history = [ loss ] # начальное значение ф-ции потери

In [20]:
# loss_history

In [21]:
from sklearn.metrics import accuracy_score

def accuracy(x,y):
    with torch.set_grad_enabled(False):
        o = model.predict(torch.Tensor(x).to(device)).cpu().numpy()
    return accuracy_score(np.argmax(y,axis=1),np.argmax(o,axis=1))

acc_history = [ accuracy(X_train,y_train) ] # начальное значение погрешности

In [22]:
# acc_history

In [23]:
from torch.utils.data import TensorDataset

# пакуем данные в формат Torch
dataset = TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train) )

In [ ]:
%%time

from tqdm import tqdm # рисует прогрессбар
from torch.utils.data import DataLoader # генератор батчей

n_epoch = 512 # количество эпох обучения
acc_min = .98 # порог минимально допустимой погрешности модели

for i in tqdm(range(n_epoch)): 
    
    for x,y in DataLoader(dataset, batch_size=len(y_train)//3, shuffle=True): # получаем батч учебных примеров
        out = model.forward(x.to(device)) # считаем выход модели
        loss = criterion( y.to(device),out ) # вычисляем значение ф-ции потери
        loss_history.append(loss.item()) # дополняем историю изменения значений ф-ции потери
        optimizer.zero_grad() # очищаем предыдущее значение градиента
        loss.backward()  # вычисляем текущее значение градиента ф-ции потери
        optimizer.step() # корректируем параметры модели
        
    acc_history.append( accuracy(X_train,y_train) ) #значение погрешности
    if acc_history[-1] > acc_min: # проверяем достижение минимального порога погрешности модели
        print('step %i/%i: loss %.03f, acc threshold %.03f reached\n'%(i+1,n_epoch,loss_history[-1],acc_min))
        break

 74%|███████▍  | 381/512 [07:32<02:35,  1.19s/it]

In [ ]:
# история изменения значений ф-ции потери
plt.plot(loss_history,label='min loss=%.3f'%(min(loss_history)))
plt.grid()
plt.legend()

In [ ]:
# история изменения значений погрешности модели
plt.plot(acc_history,label='max acc=%.3f'%(max(acc_history)),c='r')
plt.grid()
plt.legend()

In [ ]:
with torch.set_grad_enabled(False):
    o = model.predict(torch.Tensor(X_train).to(device)).cpu().numpy()
o

In [ ]:
np.argmax(o,axis=1)

In [ ]:
np.argmax(y_train,axis=1)